In [8]:
# 用于发送 HTTP 请求的库
import requests
# 用于解析 HTML 并提取信息
from bs4 import BeautifulSoup
# time 这个库主要用来记录一下时间
import time
# 在验证 ip 有效性的时候我会开多线程
from threading import Thread
'''
队列用于线程间通信。需要注意的是，得从 queue 导入 Queue。
不能从 multiprocessing 导入 Queue，这是用于进程间通信的。
虽然两者用法基本一致，但底层实现不同。
'''
from queue import Queue



class GetIp:

    def __init__(self, verify_site):
        '''
        类初始化。

        :param str verify_site: 用于对代理ip进行测试的网站，可以填入要爬的网站链接。
        '''
        # proxy_list 保存从西刺爬下来的代理
        self.proxy_list = []
        # success_list 保存验证通过的代理
        self.success_list = []
        self.site = verify_site

    def get_proxies(self):
        '''
        从西刺首页获取 http 和 https 代理地址并存入实例变量 proxy_list。
        '''
        # 记录开始时间
        start = time.time()
        print('start crawling ip...')
        try:
            # 发送http请求，加一个 User-Agent 的请求头让服务器觉得我们不像爬虫脚本，
            # 不加 headers 也可以。
            html = requests.get('http://www.xicidaili.com/', headers={
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.79 Safari/537.36'
            })
        except:
            # 若请求失败则抛出异常，中断程序，因为获取不到网页内容，继续下去也没意义了。
            raise
        # 以下的解析部分的注释省去，各位看一下 BeautifulSoup 的文档就知道了，不是本文的重点。
        soup = BeautifulSoup(html.text, 'lxml')
        trs = soup.find('table', id='ip_list').find_all('tr', class_=['', 'odd'])[2:]
        for tr in trs:
            scheme = tr.findAll('td', class_="")[-3].get_text().lower()
            if scheme in ['http', 'https']:
                ip = tr.findAll('td', class_="")[0].get_text()
                port = tr.findAll('td', class_="")[1].get_text()
                proxy = scheme + '://' + ip + ':' + port
                self.proxy_list.append(proxy)
        # 打印运行时间
        print('[Get Proxies]total time: %.2f' % (time.time() - start))


    def run(self):
        '''
        负责获取代理，开启多线程任务对代理地址进行验证。
        '''
        # 先执行获取代理的函数，将爬下来的代理暂时保存至实例变量 proxy_list
        self.get_proxies()
        start = time.time()
        # 实例化一个队列对象
        proxy_q = Queue()
        print('Verifying proxies...')
        # workers 列表保存线程
        workers = []
        # 把爬下来的代理地址逐个放进队列
        for p in self.proxy_list:
            proxy_q.put(p)
        # 开启15个线程并放入workers
        # 最后往队列放入数量与线程一样的数字0，作为每个线程结束的标记
        for _ in range(15):
            workers.append(Thread(target=self.verify_proxies, args=(proxy_q,)))
            proxy_q.put(0)
        for w in workers:
            # 启动线程
            w.start()
        for w in workers:
            # 等待线程终止
            w.join()
        print('[verification] total time: %.2f s' % (time.time() - start))
        print('proxies verified !')


    def verify_proxies(self, proxy_q):
        '''
        验证代理有效性
        :param list proxy_q: 待验证的代理队列
        '''
        # 写一个死循环不断从传入队列中获取代理地址
        while 1:
            proxy = proxy_q.get()
            # 若获取到的值是 0，即退出循环，线程终止
            if proxy == 0:  break
            protocol = 'https' if 'https' in proxy else 'http'
            proxies = {
                protocol: proxy
            }
            try:
                # 若返回状态码不等于200、或在HTTP连接过程中发生其他错误，则抛出异常
                # 设置连接超时时间为3秒
                # self.site为初始化时设置的测试链接
                assert requests.get(self.site, timeout=3, proxies=proxies).status_code == 200
            except:
                print('[fail] %s' % proxy)
            else:
                print('[success] %s' % proxy)
                # 保存验证通过的代理
                self.success_list.append(proxy)
                
                
if __name__ == '__main__':
    #print('Test Site: {}'.format('http://www.a-hospital.com/w'))
    #g = GetIp('http://www.a-hospital.com/w')
    #g.run()
    with open('proxies1.txt', 'r') as f:
        for line in f.readlines():
            print(line.strip()) # 把末尾的'\n'删掉

        
    
#         for proxy in g.success_list:
#             f.write(proxy + '\n')

http://59.44.247.194:9797
http://124.207.82.166:8008
https://211.147.239.101:57281
http://163.125.67.145:9797
http://211.162.70.229:3128
http://112.85.169.82:9999
http://125.126.208.81:9999
http://112.85.168.92:9999
https://163.204.244.100:9999
https://61.142.72.154:30074
https://180.164.24.165:53281
